In [1]:
import os
import sys
from dotenv import load_dotenv
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import pickle
import codecs

scripts_path = os.getcwd()
env_file_path = os.path.join(scripts_path, 'env.txt')
load_dotenv(env_file_path)
main_path = os.getenv("MAIN_PATH")

sys.path.append(main_path)
data_path = os.path.join(main_path, 'data')
models_path = os.path.join(main_path, 'models','DS06G_XXcoefficientiModelli')
models_xgb_path = os.path.join(main_path, 'models','DS06B_XXmodelCorporate')

In [2]:
samples_file = os.path.join(data_path, 'EW_BusinessXai_v2.xlsx')
samples = pd.read_excel(samples_file, sheet_name = 'EW_MODULI_SCORE_CORPORATE')
scores = pd.read_excel(samples_file, sheet_name = 'EW_SCORE_CORPORATE').set_index('ID')
kpi_map = pd.read_excel(os.path.join(data_path, 'mapping descrizioni KPI_v6.1.xlsx'))

kpi_map = kpi_map.rename(columns={'ID':'KPI'})
kpi_map = kpi_map[kpi_map.NOME_COLONNA_OUTPUT=='VALORE'].drop(['TIPOLOGIA_OUTPUT','COL_TO_TRANSPOSE','VALUE','CONSTANT_TO_ADD','TO_DROP','DESCRIZIONE','DATA_INIZIO_VALIDITA','DATA_FINE_VALIDITA'], axis = 1)
kpi_map = kpi_map[(kpi_map.KPI.isin(samples.KPI_CD.unique()))].drop(['NOME_COLONNA_OUTPUT', 'TABELLA'], axis = 1)
kpi_map = samples.loc[:,['MODULO_DS', 'KPI_CD']].drop_duplicates().set_index('KPI_CD').join(kpi_map.set_index('KPI')).reset_index().rename(columns={'KPI_CD':'KPI'})

data = samples.pivot(index = 'ID', columns = 'KPI_CD', values='VALORE_QT')

descriptions = pd.read_excel(os.path.join(data_path, 'logit_weights.xlsx'), sheet_name='weights')
descriptions = descriptions.set_index('Variabile').join(kpi_map.drop('MODULO_DS', axis=1).set_index('CAMPO'), how = 'inner').reset_index().sort_values(['MODELLO','MODULO_DS']).set_index('KPI')
kpi_descriptions = descriptions.loc[:,'Descrizione'].to_dict()
kpi_name = descriptions.loc[descriptions.index.dropna(),'Variabile'].to_dict()
name_kpi = {name:kpi for kpi,name in kpi_name.items()}

integration_cols = ['PD_MEDIA_UFFICIALE_M1_QT','PREVISIONE_WORST_QT']
integration_data = scores.loc[:,integration_cols].rename(columns = {'PD_MEDIA_UFFICIALE_M1_QT':'binned_PD_MEDIA_UFFICIALE_M1','PREVISIONE_WORST_QT':'binned_VAR_PREVISIONE_4_0_base_oss'})

In [3]:
slicer = pd.IndexSlice
parquet_files = os.listdir(models_path)[:-1]
models = pd.concat([pd.read_parquet(os.path.join(models_path, pqt_file)) for pqt_file in parquet_files]).reset_index(drop = True)
models['value_type'] = models.apply(lambda x: x.NAME.split('_')[0], axis = 1)

In [4]:
## CONSIDERO CORPORATE
models_corporate = models[models.MODELLO == 'Corporate']

medie_corporate = models_corporate[models_corporate.value_type == 'media']
medie_corporate['NAME'] = medie_corporate.NAME.str.replace('media_','')
pivot_medie_corporate = medie_corporate.pivot(index = ['MODELLO','MODULO','NAME'], values='VALUE', columns='value_type')

models_features_corporate = models_corporate.drop(medie_corporate.index).set_index(['MODELLO','MODULO','NAME'])

logit_models_corporate = models_features_corporate.join(pivot_medie_corporate, how = 'left').loc[slicer['Corporate',['ANDAMENTALE_INTERNO','BILANCI','CR','INTEGRAZIONE_SCORE_MODULI', 'INTEGRAZIONE_INFO']], :]
logit_models_corporate = logit_models_corporate.droplevel(0)

C:\Users\GiorgioBarazzetta\AppData\Local\Temp\ipykernel_13416\3230101557.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medie_corporate['NAME'] = medie_corporate.NAME.str.replace('media_','')


In [5]:
import shap

class XGBoost:
    def __init__(self, df):
        modelpickle = df['model'].iloc[0]
        model = pickle.loads(codecs.decode(modelpickle.encode(), "base64"))
        self.output_function = model
    
    def predict(self, X):
        pred = self.output_function.predict_proba(X)[:,1]
        pred = pd.Series(pred, index = X.index)
        pred = np.log(pred/(1- pred))
        return pred
    
    def explain(self, X):
        explainer = shap.TreeExplainer(model=self.output_function)
        explanations = explainer.shap_values(X.values)
        explanations = pd.DataFrame(explanations, columns=X.columns, index = X.index)
        return explanations



class Logit:
    def __init__(self, betas, intercept, averages):
        self.betas = betas
        self.intercept = intercept
        self.averages = averages
    
    def predict(self, X):
        log_odds_ratio  = X.transpose().apply(lambda x: (x*self.betas).sum()).dropna() + self.intercept
        # pred = log_odds_ratio.apply(lambda x: 1/(1+np.exp(-1*x)))
        return log_odds_ratio
    
    def explain(self, X):
        explanations = X.transpose().apply(lambda x: (x - self.averages)*self.betas).dropna().transpose()
        return explanations
    
    def predict_proba(self,X):
        log_odds_ratio = self.predict(X)
        pred = log_odds_ratio.apply(lambda x: 1/(1+np.exp(-1*x)))
        return pred

class Binner:
    def __init__(self, table):
        tables_rows = list(table.itertuples(index = False, name = None))
        self.bins_mapper = {pd.IntervalIndex.from_tuples([(x[0], x[1])], closed='left')[0]: x[2] for x in tables_rows}
        self.bins = pd.IntervalIndex.from_tuples([(x[0], x[1]) for x in tables_rows], closed = 'left')
    
    def transform(self, X):
        binned_X = pd.Series(pd.cut(X.values.reshape(-1), self.bins), index = X.index).replace(self.bins_mapper).astype(float)
        return binned_X

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [6]:
transformation_path = os.path.join(main_path, 'models','DS06G_XX0BinnedScoresCorp')
transformation_files = os.listdir(transformation_path)[:-1]
transformation = pd.concat([pd.read_parquet(os.path.join(transformation_path, pqt_file)) for pqt_file in transformation_files]).reset_index(drop = True)
transformation = transformation.rename(columns = {'feature':'MODULO'})
map_score = {'score_bil':'BILANCI', 'score_AI': 'ANDAMENTALE_INTERNO', 'score_cr':'CR', 'score_trans': 'TRANSAZIONALE'}
transformation = transformation.replace(map_score)

In [7]:
def create_report(data, intercept, betas, averages, predictions, explanations,  binned_predictions, pc_explanations, modulo ):
    melted_intercept = pd.DataFrame(intercept, index = data.index, columns = ['kpi_value'])
    melted_intercept['kpi'] = 'intercept'
    melted_intercept['beta'] = 1
    melted_data  = data.melt(ignore_index=False, var_name='kpi',value_name='kpi_value').sort_index().reset_index().set_index('kpi')
    melted_data = melted_data.join(betas.to_frame('beta'), how = 'inner').join(averages.to_frame('average')).reset_index().set_index('ID')
    melted_data = pd.concat([melted_data,melted_intercept])
    melted_data['modulo'] = modulo
    melted_explanations = explanations.melt(ignore_index=False, var_name=['kpi'], value_name= 'contrib').sort_index().set_index('kpi', append = True)#.drop('modulo', axis = 1)
    ff = melted_data.set_index('kpi', append = True).join(melted_explanations)

    if isinstance(pc_explanations, pd.DataFrame):
        melted_pc_explanations =  pc_explanations.melt(ignore_index=False, var_name=['kpi'], value_name= 'pc_contrib').sort_index().set_index('kpi', append = True)#.drop('modulo', axis = 1)
        ff = ff.join(melted_pc_explanations)
    ff = ff.reset_index(level = 1).join(predictions.to_frame('score'))
    
    if isinstance(binned_predictions, pd.Series):
        print('here')
        ff = ff.join(binned_predictions.to_frame('binned_score'))
    return ff

In [8]:
contribs = []
pc_contribs = []
preds = []
binned_preds = []
renamed_data = data.rename(columns=kpi_name)
binned_score_map = {'BILANCI': 'binned_score_bil', 
                    'ANDAMENTALE_INTERNO': 'binned_score_AI', 
                    'CR': 'binned_score_cr', 
                    'TRANSAZIONALE':'binned_score_trans' }
report = []

for modulo in ['ANDAMENTALE_INTERNO', 'BILANCI', 'CR']:
    intercept = logit_models_corporate.loc[slicer[modulo, 'intercetta'], 'VALUE']
    betas = logit_models_corporate.loc[slicer[modulo], 'VALUE'].drop('intercetta')
    averages = logit_models_corporate.loc[slicer[modulo], 'media'].dropna()
    model = Logit(betas, intercept, averages)

    explanations = model.explain(renamed_data)
    explanations.columns = pd.MultiIndex.from_product([[binned_score_map[modulo]], explanations.columns])
    pc_explanations = (explanations.transpose()/explanations.sum(1)).transpose()

    predictions = model.predict(renamed_data)

    current_transformation = transformation[transformation.MODULO == modulo].iloc[:,1:4]
    binned_predictions = Binner(current_transformation).transform(predictions)

    contribs += [explanations]
    pc_contribs += [pc_explanations]
    preds += [predictions.to_frame(modulo)]
    binned_preds += [binned_predictions.to_frame(binned_score_map[modulo])]
    report += [create_report(renamed_data, intercept, betas, averages, predictions, explanations.droplevel(0,axis = 1), binned_predictions, pc_explanations.droplevel(0,axis = 1), modulo)]



## TRANSAZIONALE - CALCOLO
trans_features = kpi_map[kpi_map['MODULO_DS'] == 'Transazionale'].CAMPO.values.tolist()
trans_data = renamed_data.loc[:, trans_features]
files = os.listdir(models_xgb_path)[:-1]
pd_xgb_model = pd.read_parquet(os.path.join(models_xgb_path, files[0]))
xgboost_model = XGBoost(pd_xgb_model)

trans_explanations = xgboost_model.explain(trans_data)
trans_explanations.columns = pd.MultiIndex.from_product([[binned_score_map['TRANSAZIONALE']], trans_explanations.columns])
pc_trans_explanations = (trans_explanations.transpose()/trans_explanations.sum(1)).transpose()

trans_preds = xgboost_model.predict(trans_data)

current_transformation = transformation[transformation.MODULO == 'TRANSAZIONALE'].iloc[:,1:4]
trans_binned_predictions =  Binner(current_transformation).transform(trans_preds)

contribs += [trans_explanations]
pc_contribs += [pc_trans_explanations]
preds += [trans_preds.to_frame('TRANSAZIONALE')]
binned_preds += [trans_binned_predictions.to_frame(binned_score_map['TRANSAZIONALE'])]

## TRANSAZIONALE - REPORT
melted_data = trans_data.melt(ignore_index = False, var_name='kpi', value_name='kpi_value').set_index('kpi', append = True)
melted_explanations = trans_explanations.melt(ignore_index=False, value_name='contrib', var_name=['modulo','kpi']).drop('modulo',axis = 1).set_index('kpi', append = True)
melted_pc_explanations = pc_trans_explanations.melt(ignore_index=False, value_name='pc_contrib', var_name=['modulo','kpi']).drop('modulo',axis = 1).set_index('kpi', append = True)
melted_explanations = melted_explanations.join(melted_pc_explanations)
trans_report = melted_data.join(melted_explanations).reset_index(level = 1)
trans_report['modulo'] = 'TRANSAZIONALE'
trans_report = trans_report.join(trans_preds.to_frame('score')).join(trans_binned_predictions.to_frame('binned_score'))

## INTEGRAZIONE - INPUT
contribs = pd.concat(contribs, axis = 1)
pc_contribs = pd.concat(pc_contribs, axis = 1)
preds = pd.concat(preds, axis = 1)
binned_preds = pd.concat(binned_preds, axis = 1)

## INTEGRAZIONE SCORE MODULI - CALCOLO
intercept_integrated = logit_models_corporate.loc[slicer['INTEGRAZIONE_SCORE_MODULI', 'intercetta'], 'VALUE']
betas_integrated = logit_models_corporate.loc[slicer['INTEGRAZIONE_SCORE_MODULI'], 'VALUE'].drop('intercetta')
averages_integrated = logit_models_corporate.loc[slicer['INTEGRAZIONE_SCORE_MODULI'], 'media'].dropna()
model_integrated = Logit(betas_integrated, intercept_integrated, averages_integrated)

explanations_integrated = model_integrated.explain(binned_preds)
predictions_integrated = model_integrated.predict(binned_preds).to_frame('score_integrated')

## INTEGRAZIONE SCORE MODULI - REPORT
report_integrated = create_report(binned_preds, intercept_integrated, betas_integrated, averages_integrated, predictions_integrated.squeeze(), explanations_integrated, binned_predictions = None, pc_explanations = None, modulo = 'INTEGRAZIONE')

# ## INTEGRAZIONE INFO
intercept_ew = logit_models_corporate.loc[slicer['INTEGRAZIONE_INFO', 'intercetta'], 'VALUE']
betas_ew = logit_models_corporate.loc[slicer['INTEGRAZIONE_INFO'], 'VALUE'].drop('intercetta')
averages_ew = logit_models_corporate.loc[slicer['INTEGRAZIONE_INFO'], 'media'].dropna()
model_ew = Logit(betas_ew, intercept_ew, averages_ew)

ew_dataset = pd.concat([predictions_integrated, integration_data],axis = 1)

explanations_ew = model_ew.explain(ew_dataset)
predictions_ew = model_ew.predict(ew_dataset).to_frame('score_ew')
proba_ew = model_ew.predict_proba(ew_dataset)

# ## INTEGRAZIONE INFO - REPORT
report_info = create_report(ew_dataset, intercept_ew, betas_ew, averages_ew, proba_ew, explanations_ew, binned_predictions = None, pc_explanations = None, modulo = 'INFO')

# ### BACKWARD CONTRIBUTIONS --> -1 perchè WOE e log-odds modelli per modulo sono inversamente legati, ma solo se continuo con la catena di contributi
scores_to_ew_contributions = betas_ew.loc['score_integrated']*explanations_integrated
features_to_scores = pc_contribs.apply(lambda x: x.transpose()*scores_to_ew_contributions.loc[:,x.name[0]].transpose())
explanations_integration = explanations_ew.loc[:,integration_data.columns]
explanations_integration.columns = pd.MultiIndex.from_product([['INTEGRAZIONE_INFO'], explanations_integration.columns])
features_to_scores = pd.concat([features_to_scores, explanations_integration], axis = 1)

here
here
here


In [9]:
results_path = os.path.join(main_path,'results')
results_file_path = os.path.join(results_path,'models_results.xlsx')
contributions_file_path  = os.path.join(results_path, 'contributions_results.xlsx')

concatenated_report = pd.concat(report+[trans_report]).set_index(['modulo','kpi'],append = True)

ri = report_integrated.copy()
ri = ri.replace({val:key for key,val in binned_score_map.items()}).set_index('kpi', append = True)
ri.index.names = ['ID','modulo']
concatenated_report = concatenated_report.reset_index(level = 2).join(ri.drop('modulo', axis = 1), rsuffix = '_INTEGRAZIONE')

legacy_report = pd.concat([report_integrated.join(report_info[report_info.kpi=='score_integrated'], rsuffix = '_EW'), report_info]).set_index('modulo',append = True).sort_index(ascending=[True,False])

In [ ]:
concatenated_report.reset_index(level = 1).join(report_info[report_info.kpi =='score_integrated'], rsuffix = '_EW').to_excel(os.path.join(results_path, 'test.xlsx'))

In [10]:
ew_part = report_info.drop(['modulo','score'], axis = 1).rename(columns = {'kpi': 'join_kpi'}).set_index('join_kpi', append = True)

integrated_part = report_integrated.drop(['modulo','score'], axis = 1)
integrated_part['join_kpi'] = 'score_integrated'
melted_contrib_to_ew = (betas_ew.loc['score_integrated']*explanations_integrated).melt(ignore_index = False, var_name = ['kpi'], value_name = 'contrib_to_ew_INTEGRAZIONE')#.drop('modulo', axis = 1)
integrated_part = integrated_part.set_index('kpi', append = True).join(melted_contrib_to_ew.set_index('kpi', append = True)).reset_index(level = 1)
integrated_part = integrated_part.set_index('join_kpi', append = True)

ew_integrated_part = ew_part.join(integrated_part, lsuffix = '_EW', rsuffix = '_INTEGRAZIONE')
ew_integrated_part = ew_integrated_part.reset_index(level = 1).rename(columns = {'kpi':'join_kpi', 'join_kpi':'kpi_EW'}).set_index('join_kpi', append = True)

models_part = pd.concat(report+[trans_report]).drop(['score','binned_score'], axis = 1)
models_part['join_kpi'] = models_part.modulo.replace(binned_score_map)
models_part = models_part.set_index('join_kpi', append = True)
models_part.columns = [x+'_MODELS' for x in models_part.columns]
melted_contrib_scores_to_ew = features_to_scores.drop('INTEGRAZIONE_INFO', axis = 1).melt(ignore_index = False, var_name = ['join_kpi','kpi_MODELS'], value_name = 'contrib_to_ew_MODELS').set_index(['join_kpi','kpi_MODELS'], append = True)
models_part = models_part.set_index('kpi_MODELS',append = True).join(melted_contrib_scores_to_ew)
all_contribs = ew_integrated_part.join(models_part.reset_index(level = 2)).set_index(['modulo_MODELS', 'kpi_MODELS','kpi_EW'], append = True)

dropping on a non-lexsorted multi-index without a level parameter may impact performance.


In [ ]:
ew_all_contribs = all_contribs.loc[:,[col for col in all_contribs.columns if '_EW' in col]] 
ew_all_contribs.columns = pd.MultiIndex.from_product([['EW'],[x.replace('_EW','') for x in ew_all_contribs.columns ]])
integrazione_all_contribs = all_contribs.loc[:,[col for col in all_contribs.columns if '_INTEGRAZIONE' in col]] 
integrazione_all_contribs.columns = pd.MultiIndex.from_product([['INTEGRAZIONE'],[x.replace('_INTEGRAZIONE','') for x in integrazione_all_contribs.columns ]])
models_all_contribs = all_contribs.loc[:,[col for col in all_contribs.columns if '_MODELS' in col]]
models_all_contribs.columns = pd.MultiIndex.from_product([['MODELS'],[x.replace('_MODELS','') for x in models_all_contribs.columns ]])

pd.concat([ew_all_contribs, integrazione_all_contribs, models_all_contribs], axis = 1).to_excel(os.path.join(results_path, 'contribs_test2.xlsx'))

In [11]:
### save
explanations_ew.columns = pd.MultiIndex.from_product([['INTEGRAZIONE_INFO'],explanations_ew.columns])
explanations_integrated.columns = pd.MultiIndex.from_product([['INTEGRAZIONE_SCORE_MODULI'],explanations_integrated.columns])
expl = pd.concat([
    contribs.melt(ignore_index=False, var_name=['modulo','variable_name']),
    explanations_ew.melt(ignore_index=False, var_name=['modulo','variable_name']),
    explanations_integrated.melt(ignore_index=False, var_name=['modulo','variable_name'])
])


In [ ]:
writer = pd.ExcelWriter(os.path.join(results_path,'contribs.xlsx'))
expl.to_excel(writer, sheet_name = 'models_contribs')
overall_contribs = features_to_scores.melt(ignore_index=False, var_name = ['modulo','kpi_name'], value_name = 'contrib').set_index('modulo', append = True).sort_index(ascending = [True,False])
overall_contribs.to_excel(writer, sheet_name = 'overall_contribs')
writer.close()

### INTERPRETAZIONE RISULTATI

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import openai

base_llm = ChatOpenAI(model="gpt-4", temperature = 0, openai_api_key= os.environ.get('OPENAI_API_KEY'))

The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.


In [30]:
variables_desc = pd.read_excel(os.path.join(data_path, 'logit_weights.xlsx'), sheet_name='weights')
client_id = 1
desc = variables_desc[(variables_desc.MODULO_DS!='Integrazione') & (-variables_desc.Variabile.isin(['intercept','Intercept','score_integrated']))].loc[:,['Variabile','Descrizione']].set_index('Variabile').join(features_to_scores.round(3).loc[client_id].droplevel(0).to_frame('importanza'))
mkd_table = desc.reset_index().drop('Variabile', axis = 1).to_markdown(index=False)

In [14]:
variables_listed = ' '.join(descriptions.loc[:, ['Variabile','Descrizione']].apply(lambda x: '\n -' +' : '.join(x.tolist()), axis = 1).values.tolist())

In [ ]:
prompt = """
Sei un data scientist che lavora per una banca. La tabella compresa tra i tag <table> riporta il contributo, in termini di SHAP-value, di ogni variabile alla previsione di un tuo modello di classificazione.
Ogni variabile rappresenta una caratteristica del comportamento di un cliente.
Fornisci una spiegazione della previsione svolta dal modello cercando di dedurre il comportamento del cliente.
Concentrati sulle variabili che hanno influenzato maggiormente la previsione.

<table>
{table}
<table>
""".format(table = mkd_table)

In [ ]:
print(prompt)

In [ ]:
risp = base_llm.invoke(prompt).content
print(risp)

In [ ]:
Valuta contemporaneamente più variabili per dettagliare meglio l'operatività del cliente.


In [ ]:
prompt = """
Sei un data scientist che lavora in una banca. Il tuo capo ha ricevuto la tabella compresa tra i tag <table> in cui è indicato il contributo ogni variabile alla previsione di un modello di classificazione (Il modello vuole prevedere il rischio di insolvenza di un prestito per un cliente).
Non essendo una persona tecnica, ha bisogno che lo aiuti ad interpretare la tabella. 
Le informazioni che gli interessano sapere sono:
- le caratteristiche principali del cliente
- come si sta comportando il cliente
- quale comportamento del cliente si può dedurre combinando il contributo di più variabili assieme 

Tieni a mente che un valore positivo di contributo indica che la variabile ha alzato il rischio di insolvenza; valori negativi indicano diminuzione del rischio mentre valori prossimi allo zero indicano scarsa influenza.
Focalizzati sulle variabili con contributo assoluto maggiore.

<table>
{table}
<table>
""".format(table = mkd_table)

print(prompt)

In [ ]:
risp = base_llm.invoke(prompt).content
print(risp)

In [ ]:
prompt = """
Sei un data scientist che lavora in una banca. Il tuo capo ha ricevuto la tabella compresa tra i tag <table> in cui è indicato il contributo ogni variabile alla previsione di un modello di classificazione (Il modello vuole prevedere il rischio di insolvenza di un prestito per un cliente).
Non essendo una persona tecnica, ha bisogno che lo aiuti ad interpretare la tabella. 
Ragiona per passi: individua le caratteristiche principali del cliente considerato e come queste si relazionano al rischio di insolvenza. Successivamente, combina quando dedotto e descrivi come si sta comportando il cliente.
Concludi fornendo una spiegazione delle evidenze trovate giustificandole.

Focalizzati sulle variabili con contributo assoluto maggiore.

Tieni a mente che un valore positivo di contributo indica che la variabile ha alzato il rischio di insolvenza; valori negativi indicano diminuzione del rischio mentre valori prossimi allo zero indicano scarsa influenza.

<table>
{table}
<table>
""".format(table = mkd_table)


print(prompt)

In [ ]:
risp = base_llm.invoke(prompt).content
print(risp)

### PROMPT ANALISI

In [ ]:
prompt = """
Sei un data scientist che lavora per una banca. La tabella compresa tra i tag <table> riporta il contributo, in termini di SHAP-value, di ogni variabile alla previsione di un tuo modello di classificazione.
Ogni variabile rappresenta una caratteristica del comportamento di un cliente.

Fornisci una spiegazione della previsione svolta dal modello, secondo il seguente schema:
- Variabili più rilevanti che hanno influenzato positivamente la previsione (ordinamento descrescente).
- Variabili più rilevanti che hanno influenzato negativamente la previsione (ordinamento crescente). 
- I comportamenti del cliente che si possono dedurre analizzando le variabili che hanno contribuito maggiormente.

Concludi riassumendo le evidenze che hai trovato.

Alcune note:
- Previsione prositiva del modello indica cliente con alta rischiosità.
- Utilizza i valori shap solo per identificare quali sono le variabili più rilevanti.
- Il comportamento deve essere dedotto aggregando variabili di natura simile e rilevanti per il modello.
- Nel costruire la risposta non fare riferimento ai valori shap.
- La struttura del comportamento deve essere: dato che "variabile A" e "variabile B" sono positive allora il cliente ha questo comportamento.

<table>
{table}
<table>
""".format(table = mkd_table)


print(prompt)

In [ ]:
risp = base_llm.invoke(prompt).content
print(risp)

In [54]:
prompt = """
Sei un data scientist che lavora per una banca. La tabella compresa tra i tag <table> riporta l'importanza di ogni variabile alla previsione di un tuo modello di rischiosità.
Ogni variabile rappresenta una caratteristica del comportamento di un cliente. 
Spiega la previsione svolta dal modello secondo il seguente schema:

[Variabili rilevanti]:
- Lista variabili (massimo 10) con contributo positivo ordinate in maniera decrescente.
- Lista variabili (massimo 10) con contributo negativo ordinate in maniera crescente.

[Comportamento cliente]:
Raggruppa le variabili rilevanti con natura simile e contributo coerente in gruppi. Nomina ciascun gruppo e spiega il significato del gruppo.
- Nome: <Nome gruppo>
- Tipologia: <contributo positivo o negativo>
- Variabili: <Elenco variabili presenti>
- Criterio: <Indica il criterio utilizzato per la costruzione del gruppo>

[Conclusione]
Utilizza i gruppi come fattori per spiegare la previsione del modello.

<table>
{table}
<table>
""".format(table = mkd_table)


print(prompt)


Sei un data scientist che lavora per una banca. La tabella compresa tra i tag <table> riporta l'importanza di ogni variabile alla previsione di un tuo modello di rischiosità.
Ogni variabile rappresenta una caratteristica del comportamento di un cliente. 
Spiega la previsione svolta dal modello secondo il seguente schema:

[Variabili rilevanti]:
- Lista variabili (massimo 10) con contributo positivo ordinate in maniera decrescente.
- Lista variabili (massimo 10) con contributo negativo ordinate in maniera crescente.

[Comportamento cliente]:
Raggruppa le variabili rilevanti con natura simile e contributo coerente in gruppi. Nomina ciascun gruppo e spiega il significato del gruppo.
- Nome: <Nome gruppo>
- Tipologia: <contributo positivo o negativo>
- Variabili: <Elenco variabili presenti>
- Criterio: <Indica il criterio utilizzato per la costruzione del gruppo>

[Conclusione]
Utilizza i gruppi come fattori per spiegare la previsione del modello.

<table>
| Descrizione                   

In [55]:
risp = base_llm.invoke(prompt).content
print(risp)

[Variabili rilevanti]:

Contributo positivo:
1. Somma Utilizzato su Somma Accordato negli ultimi due mesi (0.396)
2. Somma Importo Effetti Trassati Insoluti su Importo Effetti Scaduti negli ultimi tre mesi (0.322)
3. utilizzo su accordato revoca snb (0.308)
4. utilizzo su accordato autoliquidante snb (0.267)
5. PD media ufficiale nell’ultimo mese (0.261)
6. Media pesata per importo del massimo numero giorni di sconfino negli ultimi sei mesi (0.257)
7. Flag Massimo Importo Sconfino > 100 nell'ultimo mese - cliente + responsabilità limitata (0.207)
8. sconfino medio revoca > 1000€ (0.183)
9. Misura di movimentazione del conto, che considera sia il numero che gli importi dei movimenti di conto corrente (0.174)
10. utilizzo firma finanziaria su accordato firma finanziaria (0.116)

Contributo negativo:
1. Somma Utilizzato su Somma Giacenza Minima negli ultimi due mesi (-0.1)
2. Massimo Numero di Rate Impagate negli ultimi tre mesi (altro rateali - non mutui) (-0.07)
3. Variazione tra il tas

In [58]:
prompt = """
Sei un data scientist che lavora in una banca. Il tuo capo ha ricevuto la tabella compresa tra i tag <table> in cui è indicato il contributo ogni variabile alla previsione di un modello di classificazione (Il modello vuole prevedere il rischio di insolvenza di un prestito per un cliente).
Non essendo una persona tecnica, ha bisogno che lo aiuti ad interpretare la tabella senza riferimenti ai numeri presenti. 
Ragiona per passi: 
- Elenca le variabili (massimo 10) con contributo positivo ordinate in maniera decrescente
- Elenca le variabili (massimo 10) con contributo negativo ordinate in maniera crescente
- Raggruppa le variabili rilevanti con natura simile e contributo coerente in gruppi. Nomina ciascun gruppo e spiega il significato del gruppo. Utilizza il seguente schema:
    - Nome: <Nome gruppo>
    - Tipologia: <contributo positivo o negativo>
    - Variabili: <Elenco variabili presenti>
    - Criterio: <Indica il criterio utilizzato per la costruzione del gruppo>
- Concludi riportando quali gruppi hanno alzato o diminuto il rischio di insolvenza ipotizzando il comportamento avuto dal cliente.

Tieni a mente che un valore positivo di contributo indica che la variabile ha alzato il rischio di insolvenza; valori negativi indicano diminuzione del rischio mentre valori prossimi allo zero indicano scarsa influenza.

<table>

{table}

<table>
""".format(table = mkd_table)


print(prompt)


Sei un data scientist che lavora in una banca. Il tuo capo ha ricevuto la tabella compresa tra i tag <table> in cui è indicato il contributo ogni variabile alla previsione di un modello di classificazione (Il modello vuole prevedere il rischio di insolvenza di un prestito per un cliente).
Non essendo una persona tecnica, ha bisogno che lo aiuti ad interpretare la tabella senza riferimenti ai numeri presenti. 
Ragiona per passi: 
- Elenca le variabili (massimo 10) con contributo positivo ordinate in maniera decrescente
- Elenca le variabili (massimo 10) con contributo negativo ordinate in maniera crescente
- Raggruppa le variabili rilevanti con natura simile e contributo coerente in gruppi. Nomina ciascun gruppo e spiega il significato del gruppo. Utilizza il seguente schema:
    - Nome: <Nome gruppo>
    - Tipologia: <contributo positivo o negativo>
    - Variabili: <Elenco variabili presenti>
    - Criterio: <Indica il criterio utilizzato per la costruzione del gruppo>
- Concludi rip

In [59]:
risp = base_llm.invoke(prompt).content
print(risp)

Ecco un'interpretazione della tabella senza riferimenti ai numeri:

- Le variabili con contributo positivo, ordinate in maniera decrescente, sono:
    1. Somma Utilizzato su Somma Accordato negli ultimi due mesi
    2. Media pesata per importo del massimo numero giorni di sconfino negli ultimi sei mesi
    3. Somma Importo Effetti Trassati Insoluti su Importo Effetti Scaduti negli ultimi tre mesi
    4. utilizzo su accordato revoca snb
    5. utilizzo su accordato autoliquidante snb
    6. Flag Massimo Importo Sconfino > 100 nell'ultimo mese - cliente + responsabilità limitata
    7. sconfino medio revoca > 1000€
    8. utilizzo firma finanziaria su accordato firma finanziaria
    9. Copertura dell'indebitamento
    10. Somma Importo Effetti Protestati + Richiamati su Importo Effetti Scaduti negli ultimi 3 mesi

- Le variabili con contributo negativo, ordinate in maniera crescente, sono:
    1. Somma Utilizzato su Somma Giacenza Minima negli ultimi due mesi
    2. Massimo Numero di Rat